In [ ]:
import pandas as pd
import os
import ast
import numpy as np
import pickle
from datetime import datetime

In [ ]:
path = 'C:/Users/HEN1/Projects/YELP_DATASET/DATA'
file_names = ['yelp_academic_dataset_business.json', 
              'yelp_academic_dataset_checkin.json',
              'yelp_academic_dataset_review.json', 
              'yelp_academic_dataset_tip.json', 
              'yelp_academic_dataset_user.json', 
              'yelp_academic_dataset_review_notext.csv']
files = [os.path.join(path, f) for f in file_names]

In [ ]:

df_business = pd.read_json(files[0], lines=True)
#df_checkin = pd.read_json(files[1], lines=True)
#df_tip = pd.read_json(files[3], lines=True)


In [ ]:

# An error in dataset
df_business.loc[129241,'state'] = 'BC'

In [ ]:
df_review = pd.read_json(files[2], lines=True)
df_review = df_review.drop(columns=['useful', 'funny', 'cool', 'text'])
output_file = 'yelp_academic_dataset_review_notext.csv'
output = os.path.join(path, output_file)
df_review.to_csv(output)

In [ ]:
df_review = pd.read_csv(files[5], index_col=0, low_memory=True)

In [ ]:
df_user = pd.read_json(files[4], lines=True)

In [ ]:
# Get restaurants in business dataset
df_rest = df_business[df_business['categories'].str.contains('Restaurant.*')==True].reset_index()
df_rest = df_rest.drop(columns = 'index')

In [ ]:
df_rest_name = df_rest[['business_id', 'name']]
df_rest_name

In [ ]:
df_rest = pd.concat([df_rest.drop(['attributes'], axis=1), df_rest['attributes'].apply(pd.Series)], axis=1)

In [ ]:
df_rest = df_rest.dropna(axis=1,thresh=len(df_bn1)*0.1)

In [ ]:
df_rest.info()

In [ ]:
df_rest['BusinessParking']=df_rest['BusinessParking'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)

In [ ]:
df_rest = pd.concat([df_rest.drop(['BusinessParking'], axis=1), df_rest['BusinessParking'].apply(pd.Series)], axis=1)

In [ ]:
df_rest = df_rest.drop(columns = 0)

In [ ]:
df_rest = pd.concat([df_rest.drop(['hours'], axis=1), df_rest['hours'].apply(pd.Series)], axis=1)

In [ ]:
df_rest['Ambience']=df_rest['Ambience'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)
df_rest['GoodForMeal']=df_rest['GoodForMeal'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)

In [ ]:
df_rest = pd.concat([df_rest.drop(['Ambience'], axis=1), df_rest['Ambience'].apply(pd.Series)], axis=1)
df_rest = pd.concat([df_rest.drop(['GoodForMeal'], axis=1), df_rest['GoodForMeal'].apply(pd.Series)], axis=1)

In [ ]:
df_rest['Music']=df_rest['Music'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)
df_rest = pd.concat([df_rest.drop(['Music'], axis=1), df_rest['Music'].apply(pd.Series)], axis=1)

In [ ]:
df_rest = df_rest.drop(columns = 0)

In [ ]:
df_categories_dummies = pd.Series(df_rest['categories']).str.get_dummies(',')
df_categories_dummies = df_categories_dummies.astype('category')
df_rest = pd.concat([df_rest.drop(['categories'], axis=1), df_categories_dummies], axis=1)

In [ ]:
tf_col = ['RestaurantsTableService', 'BikeParking', 'BusinessAcceptsCreditCards', 'RestaurantsReservations', 'WheelchairAccessible', 'Caters',
          'OutdoorSeating', 'RestaurantsGoodForGroups', 'HappyHour', 'BusinessAcceptsBitcoin', 'HasTV', 'DogsAllowed', 
          'RestaurantsTakeOut','RestaurantsDelivery', 'GoodForKids', 'lot', 'street', 'valet', 'validated',
          'casual', 'classy', 'divey', 'hipster', 'intimate', 'romantic', 'touristy', 'trendy', 'upscale', 'breakfast', 
          'brunch', 'dessert', 'dinner', 'latenight', 'lunch', 'background_music', 'dj', 'jukebox', 'karaoke', 'live', 'no_music', 
          'video', 'garage'] 
cat_col = list(df_rest.select_dtypes(include=['object']).columns)

In [ ]:
d = {'True': 'True', 'False': 'False', 'None':np.nan, True: 'True', False: 'False'}
for col in tf_col: 
    df_rest[col] = df_rest[col].map(d)


In [ ]:
week_col = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_rest = df_rest.drop(columns=week_col)

In [ ]:
wifi_d = {"u'free'": 'free', "'free'": 'free', "u'paid'": 'paid', "'paid'": 'paid', "u'no'": 'no', "'no'": 'no', 'None': np.nan}
df_rest['WiFi'] = df_rest['WiFi'].map(wifi_d)

In [ ]:
Alcohol_d = {"'beer_and_wine'": 'beer_and_wine', "u'beer_and_wine'": 'beer_and_wine', 
             "'full_bar'": 'full_bar', "u'full_bar'": 'full_bar', 
             "u'none'": 'no', "'none'": 'no', 
             'None': np.nan}
df_rest['Alcohol'] = df_rest['Alcohol'].map(Alcohol_d)

In [ ]:
NoiseLevel_d = {"u'average'": 'average', "'average'": 'average', 
                "u'quiet'": 'quiet', "'quiet'": 'quiet', 
                "'loud'": 'loud', "u'loud'": 'loud', 
                "u'very_loud'":'very_loud' , "'very_loud'": 'very_loud',
                'None': np.nan}
df_rest['NoiseLevel'] = df_rest['NoiseLevel'].map(NoiseLevel_d)

In [ ]:
RestaurantsAttire_d = {"'casual'": 'casual', "u'casual'": 'casual', 
                       "u'dressy'": 'dressy', "'dressy'": 'dressy', 
                       "u'formal'": 'formal', "'formal'": 'formal', 
                       'None': np.nan}
df_rest['RestaurantsAttire'] = df_rest['RestaurantsAttire'].map(RestaurantsAttire_d)

In [ ]:
RestaurantsPriceRange2_d = {1.0:1, 2.0:2, 3.0:3, 4.0:4, '1':1, '2':2, '3':3, '4':4, 'None':np.nan}
df_rest['RestaurantsPriceRange2'] = df_rest['RestaurantsPriceRange2'].map(RestaurantsPriceRange2_d)

In [ ]:
for col in tf_col: 
    df_dummies = pd.get_dummies(df_rest[col], prefix=col)
    df_dummies = df_dummies.astype('category')
    df_rest = pd.concat([df_rest.drop([col], axis=1), df_dummies], axis=1)

In [ ]:
level_col = ['WiFi', 'Alcohol', 'NoiseLevel', 'RestaurantsAttire', 'RestaurantsPriceRange2' ]

for col in level_col: 
    df_dummies = pd.get_dummies(df_rest[col], prefix=col)
    df_dummies = df_dummies.astype('category')
    df_rest = pd.concat([df_rest.drop([col], axis=1), df_dummies], axis=1)

In [ ]:
df_rest['is_open'] = df_rest['is_open'].astype('category')

In [ ]:
df_rest.info(memory_usage='deep')

In [ ]:
df_rest = df_rest.drop(columns=['name', 'address'])

In [ ]:
df_rest_van_id = df_rest[(df_rest['city']=='Vancouver') & (df_rest['state']=='BC')][['business_id']]

In [ ]:
df_review = df_review[df_review['business_id'].isin(df_rest_van_id['business_id'])]

In [ ]:
df_review = df_review.join(df_rest.set_index('business_id'), on='business_id')

In [ ]:
df_review['date'] = pd.to_datetime(df_review['date'], format='%Y%m%d %H:%M:%S')

In [ ]:
df_user = df_user[['user_id', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'elite']]
df_user['elite'] = df_user.elite.str.split(',')
df_user['yelping_since'] = pd.to_datetime(df_user['yelping_since'], format='%Y%m%d %H:%M:%S')
df_user = df_user.rename(columns={'review_count':'use_review_count'})

In [ ]:
df_review = df_review.join(df_user.set_index('user_id'), on='user_id')
is_elite = df_review.apply(lambda row: 1 if (str(row['date'].year) in row['elite']) else 0, axis =1)
is_elite = is_elite.astype('category')
df_review = pd.concat([df_review.drop(['elite'], axis=1), is_elite], axis=1)
df_review = df_review.rename(columns={0:'is_elite'})

In [ ]:
df_review = df_review.drop(columns=['review_id', 'city', 'state'])

In [ ]:
obj_col = df_review.select_dtypes(include=['object']).columns
cat = df_review[obj_col].astype('category')
df_review[obj_col] = cat

In [ ]:

output_file = 'yelp.pkl'
output = os.path.join(path, output_file)
with open(output, 'wb') as file:
    pickle.dump(df_review, file)
    